In [1]:
from labml import experiment
from python_autocomplete.train import Configs
from python_autocomplete.evaluate import evaluate, anomalies, complete, Predictor
import torch
import torch.nn
from labml import experiment, logger, lab
from labml_helpers.module import Module
from labml.logger import Text, Style
from labml.utils.pytorch import get_modules
from torch import nn
import string

from labml_helpers.datasets.text import TextDataset

In [2]:
TRAINING_RUN_UUID = '39b03a1e454011ebbaff2b26e3148b3d'
TRAINING_RUN_UUID

'39b03a1e454011ebbaff2b26e3148b3d'

In [3]:
conf = Configs()
experiment.evaluate()

In [4]:
conf_dict = experiment.load_configs(TRAINING_RUN_UUID)
conf_dict

{'batch_size': 12,
 'epochs': 32,
 'model': 'transformer_model',
 'n_layers': 6,
 'optimizer.learning_rate': 1.0,
 'optimizer.optimizer': 'Noam',
 'seq_len': 512,
 'train_loader': 'shuffled_train_loader',
 'valid_loader': 'shuffled_valid_loader'}

In [5]:
conf_dict['device.cuda_device'] = 1

In [6]:
experiment.configs(conf, conf_dict)

In [7]:
experiment.add_pytorch_models({'model': conf.model})

In [8]:
experiment.load(TRAINING_RUN_UUID)

In [9]:
experiment.start()

In [10]:
p = Predictor(conf.model, conf.text.stoi, conf.text.itos)
_ = conf.model.eval()

In [11]:
PROMPT = """from torch import nn

from labml_helpers.module import Module
from labml_nn.lstm import LSTM


class LSTM(Module):
    def __init__(self, *,
                 n_tokens: int,
                 embedding_size: int,
                 hidden_size int,
                 n_layers int):"""

In [16]:
%%time
res =  p.get_token(PROMPT)
print('"' + res + '"')

"(LSTM"
CPU times: user 386 ms, sys: 6.33 ms, total: 392 ms
Wall time: 41 ms


In [17]:
PROMPT += res
res = p.get_token(PROMPT)
print('"' + res + '"')

","


In [18]:
with open(str(lab.get_data_path() / 'sample.py'), 'r') as f:
    sample = f.read()
print(sample[-50:])

ckpoint()


if __name__ == '__main__':
    main()



In [19]:
%%time
evaluate(p, sample)

CPU times: user 2min 18s, sys: 1.17 s, total: 2min 19s
Wall time: 1min 23s


In [20]:
anomalies(p, sample)

In [21]:
sample = """import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from torchvision import datasets, transforms

from labml import lab


class Model(nn.Module):
"""

complete(p, sample, 100)